In [3]:
import re,os,time,json,random,subprocess,ollama,argparse,pandas as pd
from glob import glob
from tqdm import tqdm
from openai import OpenAI
import uuid
import argparse
import sys
import secrets
import string

def generate_secure_random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(secrets.choice(characters) for _ in range(length))
# Encoding
file_encoding = "utf-8"
# Paths
"""
when using Github Desktop to open the source directory
home path should be \'...\\Malicious_Code_Generation \'
home_path is the main directory of the project

The name of responce_title is based on the time.
""" 
source                  ="openai"#"ollama"
llm                     ="gpt-4o-mini"#"llama3.2"
home_path               = os.getcwd()
response_title          = time.strftime("%Y%m%d%H%M",time.localtime())
response_folder_path    = os.path.join(home_path,"response")
targetOS               ="Windows 11"
def Llmrequest(messages:list=[{"role":"user","content":"nothing to say"}],source:str=source,llm:str=llm):
    """ 
    目前有Openai api與Ollama 套件作為LLM來源
    """
        
    def gpt_request(llm:str="gpt-4o-mini",messages: list=[{"role":"user","content":"nothing to say"}]):
        # Get Response from GPT
        api_key = os.getenv("OPENAI_API_KEY")
        client = OpenAI(api_key = api_key)
        response = client.chat.completions.create(
        model = llm,
        messages = messages,
        temperature = 0
        )
        return response.choices[0].message.content
    def ollama_request(llm:str = "llama3.2", messages:list = [{"role":"user","content":"nothing to say"}]):
        response= ollama.chat(
        model=llm,
        messages=messages
        )
        return response['message']['content']
    if source == "openai":
        return gpt_request(llm=llm,messages=messages)
    elif source == "ollama" :
        return ollama_request(llm=llm,messages=messages)
def Llminteraction(dict_input:dict={},task:str="",outputFormatRegex:str="",display:bool=False,suffix:str=""):
    print(f"[+]Start {task}.")
    """
    """
    messages = [
        {"role": "system", "content": textLoader(txtName=f"SystemPrompt_{task}")},
        {"role": "user", "content": "<text>"+str(dict_input)+"</text>"}]
    if display:print(messages)
    output=Llmrequest(messages)
    print(f"[-]Finish {task}.")
    if outputFormatRegex != "":
        outputPattern=re.compile(r'<output>(.*?)</output>', re.DOTALL) 
        outputContent = outputPattern.findall(output)
        for i in range(len(outputContent)):
            with open(os.path.join(os.getcwd(),"response",suffix+"_"+task+"_"+str(i)+".yaml"),"w",encoding='utf-8')as f:
                f.write(outputContent[i])
        return outputContent
    
    with open(os.path.join(os.getcwd(),"response",suffix+"_"+task+".yaml"),"w",encoding='utf-8')as f:
        f.write(output)
    return output
"""
fuction : Load the prompt from thedirectory :Prompt:
"""
def textLoader(txtName:str):
    with open(os.path.join(os.getcwd(),"prompt",txtName+".txt"),"r",encoding="utf-8")as file:
        prompt=file.read()
    return prompt

In [ ]:
import os
import json
with open(os.path.join(os.getcwd(),"data","mitre_ttp_data.json"),"r",encoding='utf-8')as f:
    #print(type(f.read()))
    ttp=json.load(f)
techniques = [
    "T1005","T1007","T1016","T1021.001","T1033","T1036.004","T1046","T1047","T1049","T1053.005","T1055.002","T1059.001","T1069.001","T1071.001","T1082","T1083","T1087.001","T1105","T1112","T1204.002","T1219","T1491","T1518.001","T1547.001","T1547.009","T1548.002","T1562.001","T1564.003","T1566.001","T1567"]

for i in techniques:
    print(i)
    TTP_ID=i
    TTP_Description=ttp[i]["Description"]
    pe=ttp[i]["Procedure Examples"]
    filename=generate_secure_random_string(10)
    attackPhase=Llminteraction(dict_input={"TTP_ID": {TTP_ID},"TTP_Description": {TTP_Description},"procedure example":pe},task="attackPhaseAnalysis_COSTAR",suffix=i+"_"+filename)
    attackPattern=Llminteraction(dict_input={"attackPhase":attackPhase},task="attackPatternInferencer_COSTAR",outputFormatRegex=r'<output>(.*?)</output>',suffix=i+"_"+filename+"_COSTAR")
    for j in range(len(attackPattern)):
        logInference=Llminteraction(dict_input={"attackPattern":attackPattern[j]},task="logInferencer_COSTAR",suffix=i+"_"+filename+"_"+str(j))
        sigmarule=Llminteraction(dict_input={"logInference":logInference},task="sigmaRuleGenerator_COSTAR",suffix=i+"_"+filename+"_"+str(j))
        KQL=Llminteraction(dict_input={"sigmarule":sigmarule},task="KQLGeneration",outputFormatRegex=r'<output>(.*?)</output>',suffix=i+"_"+filename+"_COSTAR_"+str(j))

T1005
[+]Start attackPhaseAnalysis_COSTAR.
[-]Finish attackPhaseAnalysis_COSTAR.
[+]Start attackPatternInferencer_COSTAR.


### 製作偏好資料集
製作方法：利用大型語言模型「生成有特定缺陷的輸出」作為DPO所需偏好資料集中的「偏惡資料」

In [ ]:
#生成可能的缺陷面向
#依據不同的缺陷末向
import os
import json
with open(os.path.join(os.getcwd(),"data","mitre_ttp_data.json"),"r",encoding='utf-8')as f:
    #print(type(f.read()))
    ttp=json.load(f)
techniques = ["T1005"]

for i in techniques:
    print(i)
    TTP_ID=i
    TTP_Description=ttp[i]["Description"]
    pe=ttp[i]["Procedure Examples"]
    
    filename=generate_secure_random_string(10)
    #針對第一階段的提示詞生成可能的缺陷面向
    #生成一次正常的輸出
    attackPhase=Llminteraction(
        dict_input={
            "TTP_ID": {TTP_ID},
            "TTP_Description": {TTP_Description},
            "procedure example":pe},
        task="attackPhaseAnalysis_COSTAR",
        suffix=i+"_"+filename)
    DefectAspectGeneration_attackPhase=Llminteraction(
        dict_input={
            "prompt":textLoader(txtName=f"SystemPrompt_attackPhaseAnalysis_COSTAR"),
            "TTP_ID": {TTP_ID},
            "TTP_Description": {TTP_Description},
            "procedure example":pe},
        task="DefectAspectGeneration",
        suffix=i+"_"+filename)
    #

    
    DefectAspectGeneration_attackPattern=Llminteraction(
        dict_input={
            "prompt":textLoader(txtName=f"SystemPrompt_attackPatternInferencer_COSTAR"),
            "attackPhase":attackPhase},
        task="DefectAspectGeneration",
        #outputFormatRegex=r'<output>(.*?)</output>',
        suffix=i+"_"+filename+"_COSTAR")
    attackPattern=Llminteraction(
        dict_input={
            "attackPhase":attackPhase},
        task="attackPatternInferencer_COSTAR",
        #outputFormatRegex=r'<output>(.*?)</output>',
        suffix=i+"_"+filename+"_COSTAR")
    j=0
    DefectAspectGeneration_logInference=Llminteraction(
        dict_input={
            "prompt":textLoader(txtName=f"SystemPrompt_logInferencer_COSTAR"),
            "attackPattern":attackPattern[j]
            },
        task="DefectAspectGeneration",
        suffix=i+"_"+filename+"_"+str(j))
    logInference=Llminteraction(
        dict_input={"attackPattern":attackPattern[j]},
        task="logInferencer_COSTAR"
        ,suffix=i+"_"+filename+"_"+str(j))
        
    DefectAspectGeneration_sigmarule_DefectAspectGeneration=Llminteraction(
        dict_input={
            "prompt":textLoader(txtName=f"SystemPrompt_sigmaRuleGenerator_COSTAR"),
            "logInference":logInference},
        task="DefectAspectGeneration",
        suffix=i+"_"+filename+"_"+str(j))
    sigmarule=Llminteraction(
        dict_input={"logInference":logInference},
        task="sigmaRuleGenerator_COSTAR",
        suffix=i+"_"+filename+"_"+str(j))
        
    DefectAspectGeneration_KQL=Llminteraction(
        dict_input={"sigmarule":sigmarule},
        task="KQLGeneration",
        outputFormatRegex=r'<output>(.*?)</output>',
        suffix=i+"_"+filename+"_COSTAR_"+str(j))
    KQL=Llminteraction(
        dict_input={"sigmarule":sigmarule},
        task="KQLGeneration",
        outputFormatRegex=r'<output>(.*?)</output>',
        suffix=i+"_"+filename+"_COSTAR_"+str(j))

T1005
[+]Start attackPhaseAnalysis_COSTAR.
